In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/Rumour/Try")
os.getcwd()

'/content/drive/My Drive/Rumour/Try'

In [0]:
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-02-09 13:48:45--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.100.117
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.100.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz.2’

GoogleNews-vectors- 100%[===================>]   1.53G  55.6MB/s    in 29s     

2019-02-09 13:49:13 (54.8 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz.2’ saved [1647046227/1647046227]



In [0]:
import gzip
with gzip.open('GoogleNews-vectors-negative300.bin.gz','rb') as f:
  file_content=f.read()

In [0]:
open('w2v.bin','wb').write(file_content)

3644258522

In [0]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
"""
Function in this file extracts features from conversations(threads) and returns
dictionary with features
"""

import numpy as np

import nltk
import re,pickle
import help_prep_functions
from textblob import TextBlob
#from tree2branches import tree2branches
from Lexicons import false_antonyms,false_synonyms,negation_words,SpeechAct
with open('vocab2vec.pkl','rb') as r:
  d=pickle.load(r)
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}
wh_words = ['what', 'when', 'where', 'which', 'who', 'whom', 'whose', 'why','how']
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

def content_formatting(text,content):
	if text.find(content)>=0:
		return 1
	else:
		return 0

def count_lexicon(tokens,lexicon_list):
	count=0
	for token in tokens:
		if token in lexicon_list:
			count += 1
	return count

def get_speechAct_vector(text):
	sv=[]
	for key in SpeechAct.keys():
		count=0
		for verb in SpeechAct[key]:
			if verb in text.lower():
				count+=1
		sv.append(count)
	return sv
	
def extract_source_features(conversation):
    features=[]

    tw = conversation['source']
    tokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+', '',tw['text'].lower()))

    otherthreadtweets = ''
    for response in conversation['replies']:
        otherthreadtweets += ' ' + response['text']
            
    otherthreadtokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+', '',otherthreadtweets.lower()))

    source_tweet = tw['text']
    features.extend(help_prep_functions.sumw2v(tw, avg=True))
    features.append(content_formatting(source_tweet,'?'))
    features.append(content_formatting(source_tweet,'!'))
    features.append(content_formatting(source_tweet,'.'))
    features.append(content_formatting(source_tweet,'#'))
    hasurl=0
    if (content_formatting(source_tweet,'urlurlurl')>=0) or (content_formatting(source_tweet,'http') >= 0):
        hasurl = 1
    features.append(hasurl)
    haspic=0
    if (content_formatting(source_tweet,'picpicpic') >= 0) or (content_formatting(source_tweet,'pic.twitter.com') >= 0) or (content_formatting(source_tweet,'instagr.am') >= 0):
        haspic=1
    features.append(haspic)
    negation_count= 0
    for negationword in negation_words:
        if negationword in tokens:
            negation_count += 1
    features.append(negation_count)
    features.append(len(source_tweet))	#Chararcter count
    features.append(len(nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','', source_tweet.lower()))))	#Word count
    swearwords = []
    with open('data/badwords.txt', 'r') as f:
        for line in f:
            swearwords.append(line.strip().lower())
    hasswearwords = 0
    for token in tokens:
        if token in swearwords:
            hasswearwords += 1
    features.append(hasswearwords)
    uppers = [l for l in source_tweet if l.isupper()]
    features.append(float(len(uppers))/len(source_tweet))	#Capital Ratio
    #feature_dict['avgw2v'] = help_prep_functions.sumw2v(tw, avg=True)
    features.append(check_pos_tag(source_tweet, 'noun'))
    features.append(check_pos_tag(source_tweet, 'verb'))
    features.append(check_pos_tag(source_tweet, 'adj'))
    features.append(check_pos_tag(source_tweet, 'adv'))
    features.append(check_pos_tag(source_tweet, 'pron'))
    features.append(count_lexicon(tokens,false_synonyms)) #src
    features.append(count_lexicon(tokens,false_antonyms))#src
    features.append(count_lexicon(otherthreadtokens,false_synonyms)) #Thread
    features.append(count_lexicon(otherthreadtokens,false_antonyms)) #Thread
    features.append(count_lexicon(tokens,wh_words))
    features.append(count_lexicon(otherthreadtokens,wh_words)) #Thread
    
    #features.extend(get_speechAct_vector(source_tweet))
    #features.extend(get_speechAct_vector(otherthreadtweets))

    return features


def extract_thread_features_incl_response(conversation):
    source_features = extract_source_features(conversation)
    source_features.append(1)	#Source_tweets Flag
    srctokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+', '',conversation['source']['text'].lower()))
    fullthread_featdict = {}
    src=np.asarray(source_features)
    fullthread_featdict[conversation['source']['id_str']] = src
    for tw in conversation['replies']:
        features=[]
        tokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+', '',tw['text'].lower()))
        otherthreadtweets = ''
        otherthreadtweets += conversation['source']['text'] 
   
            
        for response in conversation['replies']:
            otherthreadtweets += ' ' + response['text']
                
        otherthreadtokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+', '',otherthreadtweets.lower()))
        branches = conversation['branches']
        for branch in branches:
            if tw['id_str'] in branch:
                if branch.index(tw['id_str'])-1 == 0:
                    prevtokens = srctokens
                else:
                    prev_id = branch[branch.index(tw['id_str'])-1]
                    for ptw in conversation['replies']:
                        if ptw['id_str'] == prev_id:
                            prevtokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','',ptw['text'].lower()))
                            break
            else:
                prevtokens = []
            break
        reply_tweet = tw['text']
        features.extend(help_prep_functions.sumw2v(tw, avg=True))
        features.append(content_formatting(reply_tweet,'?'))
        features.append(content_formatting(reply_tweet,'!'))
        features.append(content_formatting(reply_tweet,'.'))
        features.append(content_formatting(reply_tweet,'#'))
        hasurl=0
        if (content_formatting(reply_tweet,'urlurlurl')>=0) or (content_formatting(reply_tweet,'http') >= 0):
            hasurl = 1
        features.append(hasurl)
        haspic=0
        if (content_formatting(reply_tweet,'picpicpic') >= 0) or (content_formatting(reply_tweet,'pic.twitter.com') >= 0) or (content_formatting(reply_tweet,'instagr.am') >= 0):
            haspic = 1
        features.append(haspic)
        negation_count= 0
        for negationword in negation_words:
            if negationword in tokens:
                negation_count += 1
        features.append(negation_count)
        features.append(len(reply_tweet))	#Chararcter count
        features.append(len(nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','', reply_tweet.lower()))))	#Word count
        swearwords = []
        with open('data/badwords.txt', 'r') as f:
            for line in f:
                swearwords.append(line.strip().lower())
                hasswearwords = 0
                for token in tokens:
                    if token in swearwords:
                        hasswearwords += 1
        features.append(hasswearwords)
        uppers = [l for l in reply_tweet if l.isupper()]
        if len(reply_tweet)==0:
          features.append(0)
          print ("LOL")
        else:
          features.append(float(len(uppers))/len(reply_tweet))	#Capital Ratio
        #feature_dict['avgw2v'] = help_prep_functions.sumw2v(tw, avg=True)
        features.append(check_pos_tag(reply_tweet, 'noun'))
        features.append(check_pos_tag(reply_tweet, 'verb'))
        features.append(check_pos_tag(reply_tweet, 'adj'))
        features.append(check_pos_tag(reply_tweet, 'adv'))
        features.append(check_pos_tag(reply_tweet, 'pron'))
        features.append(count_lexicon(tokens,false_synonyms)) #rep
        features.append(count_lexicon(tokens,false_antonyms))#rep
        features.append(count_lexicon(otherthreadtokens,false_synonyms)) #Thread
        features.append(count_lexicon(otherthreadtokens,false_antonyms)) #Thread
        features.append(count_lexicon(tokens,wh_words))
        features.append(count_lexicon(otherthreadtokens,wh_words)) #Thread
        #features.extend(get_speechAct_vector(reply_tweet))
        #features.extend(get_speechAct_vector(otherthreadtweets))	#Thread
        features.append(0)
        
        #feature_dict['avgw2v'] = help_prep_functions.sumw2v(tw, avg=True)
        rep=np.asarray(features)
		
        fullthread_featdict[tw['id_str']] = rep	
    return fullthread_featdict


In [0]:

#from extract_thread_features_W2V import extract_thread_features_incl_response
import help_prep_functions
import numpy as np
import os,json
from keras.preprocessing.sequence import pad_sequences

label_veracity={"true":0,"false":1,"unverified":2}

def convert_to_array_of_branches(thread_feature_dict, conversation):
    thread_features_array = []

    branches = conversation['branches']

    for branch in branches:
        branch_rep = np.zeros(323)
        for twid in branch:
            if twid in thread_feature_dict.keys():
                tweet_rep = thread_feature_dict[twid]
                #print (tweet_rep.shape)
                branch_rep=np.vstack((branch_rep,tweet_rep))
        if branch_rep.shape[0]>=1:
            #branch_rep = np.asarray(branch_rep)
            thread_features_array.append(branch_rep)
     
    return thread_features_array

def main():
    
    path = 'Saved_dataset_train_all'
    folds = {}
    with open('dataset_train_all.json','r') as r:
        folds=json.load(r)

    
    for fold in folds.keys():
        
        feature_fold = []
        labels = []
        ids = []
        for conversation in folds[fold]:
            
            thread_feature_dict = extract_thread_features_incl_response(conversation)
            thread_features_array = convert_to_array_of_branches(thread_feature_dict, conversation)
            feature_fold.extend(thread_features_array)
            for i in range(len(thread_features_array)):
                labels.append(label_veracity[conversation['veracity']])
                ids.append(conversation['id'])
        
        if feature_fold!=[]:

            feature_fold = pad_sequences(feature_fold, maxlen=None,dtype='float32',padding='post',truncating='post', value=0.)
            labels = np.asarray(labels)
            path_fold = os.path.join(path, fold)
            if not os.path.exists(path_fold):
                os.makedirs(path_fold)
            np.save(os.path.join(path_fold, 'train_array'), feature_fold)
            np.save(os.path.join(path_fold, 'ids'), ids)
            np.save(os.path.join(path_fold, 'labels'), labels)



if __name__ == '__main__':
    main()


Using TensorFlow backend.


In [0]:
import json
veracity_labels={0:"true",1:"false",2:"unverified"}
sdqc_labels={0:"support",1:"deny",2:"query",3:"comment"}
def save_output(idsa, predictionsa, idsb, predictionsb, confidenceb):
    
    subtaskaenglish = {}
    subtaskbenglish = {}
    
    for i in range(len(idsa)):
      subtaskaenglish[idsa[i]]=sdqc_labels[predictionsa[i]]    

    for i in range(len(idsb)):
      subtaskbenglish[idsb[i]]=veracity_labels[predictionsb[i]]
    answer = {}
    answer['subtaskaenglish'] = subtaskaenglish
    answer['subtaskbenglish'] = subtaskbenglish
    
    answer['subtaskadanish'] = {}
    answer['subtaskbdanish'] = {}
    
    answer['subtaskarussian'] = {}
    answer['subtaskbrussian'] = {}

    #print (answer)
    #print (answer["subtaskbenglish"])
    with open("answer_veracity_1002.json", 'w') as f:
        json.dump(answer, f)


In [0]:


import numpy as np
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM
from keras.layers import TimeDistributed, Masking
from keras import optimizers
from keras import regularizers
from sklearn.metrics import accuracy_score
import os
from keras.utils.np_utils import to_categorical
from branch2treelabels import branch2treelabels
#from Save_Output import save_output,convertsave_competitionformat


def LSTM_model_veracity(x_train, y_train, x_test):
    model = Sequential()
    num_features = x_train.shape[2]
    model.add(Masking(mask_value=0., input_shape=(None, num_features)))
    model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
    model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2,return_sequences=False))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax',activity_regularizer=regularizers.l2(3e-4)))
    adam = optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999,epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(x_train, y_train,epochs=100, shuffle=True, class_weight=None)
    model_json = model.to_json()
    with open("model_1002.json", "w") as json_file:
      json_file.write(model_json)
    model.save_weights("model_1002.h5")
    print("Saved model to disk")
    
    json_file = open('model_1002.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights("model_1002.h5")
    print("Created LSTM Model")
    pred_probabilities = model.predict(x_test)
    confidence = np.max(pred_probabilities, axis=1)
    Y_pred = model.predict_classes(x_test)
    return Y_pred, confidence

def main():
    x_train = np.load(os.path.join('preprocessing/Saved_dataset_train_all','train/train_array.npy'))
    y_train = np.load(os.path.join('preprocessing/Saved_dataset_train_all', 'train/labels.npy'))
    y_train = to_categorical(y_train, num_classes=None)
    x_test = np.load(os.path.join('preprocessing/Saved_dataset_train_all', 'train/train_array.npy'))
    y_test = np.load(os.path.join('preprocessing/Saved_dataset_train_all', 'train/labels.npy'))
    ids_test = np.load(os.path.join('preprocessing/Saved_dataset_train_all','train/ids.npy'))
    y_pred, confidence = LSTM_model_veracity(x_train, y_train, x_test)
    trees, tree_prediction, tree_label,veracity_confidence = branch2treelabels(ids_test, y_test,y_pred,confidence)
    """
    for i in range(len(veracity_confidence)):
        if tree_prediction[i]==2:
            #veracity_confidence[i]=0
            tree_prediction[i]=1
    """
    dummy=[]
    save_output(dummy,dummy,trees,tree_prediction,veracity_confidence)
    print ("Charliehedbo event is the Test Data")
    print (trees)
    print ("Predictions",(tree_prediction))
    print ("Labels",(tree_label))
    print ("Confidence",(veracity_confidence))
    print ("Accuracy :",(accuracy_score(tree_label,tree_prediction)))
    from sklearn.metrics import classification_report
    print(classification_report(tree_label,tree_prediction,target_names=["true","false","unverified"]))

    
    import scikitplot as skplt
    import matplotlib.pyplot as plt
    skplt.metrics.plot_confusion_matrix(tree_label,tree_prediction)
    plt.show()



main()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
4013/4013 [==============================] - 53s 13ms/step - loss: 1.0864 - acc: 0.4067
Epoch 2/100
4013/4013 [==============================] - 51s 13ms/step - loss: 1.0677 - acc: 0.4448
Epoch 3/100
4013/4013 [==============================] - 51s 13ms/step - loss: 1.0446 - acc: 0.4941
Epoch 4/100
4013/4013 [==============================] - 52s 13ms/step - loss: 1.0264 - acc: 0.4902
Epoch 5/100
4013/4013 [==============================] - 51s 13ms/step - loss: 1.0021 - acc: 0.4966
Epoch 6/100
4013/4013 [==============================] - 50s 13ms/step - loss: 0.9852 - acc: 0.5088
Epoch 7/100
4013/4013 [==============================] - 50s 13ms/step - loss: 0.9732 - acc: 0.5213
Epoch 8/100
4013/4013 [==============================] - 50s 13ms

ModuleNotFoundError: ignored